# Ensemble Code 3  - Regression Cross Validation

### 필요한 모듈을 불러온다

In [1]:
import numpy as np
import copy
import pandas as pd
import matplotlib.pyplot as plt
import collections
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
print("Module Ready!")

Module Ready!


### 사용할 데이터를 불러오며 데이터 전처리함
> ##### 1. Fold_Value는 CV Fold갯수를 의미하며 3으로 입력
> ##### 2. 사용할 데이터 mortgage를 불러오며 standardization함
> ##### 3.난수 고정하여  training:test = 8:2 사용

In [2]:
#############################
FOLD_VALUE = 3
RANDOM_STATE = 1026
#############################

def mean_absolute_percentage_error(Data):
    return(np.mean(np.abs(Data[:, 1]-Data[:, 0])/Data[:, 1]))

def Valid_Index(Data,Name):
    return(pd.DataFrame(pd.Series({'MAE':mean_absolute_error(Data[:, 0], Data[:, 1]),
               'MSE': mean_squared_error(Data[:, 0], Data[:, 1]),
               'MAPE': mean_absolute_percentage_error(Data),
               'Model': Name})).transpose())   


# 사용할 Personal Loan 데이터셋을 불러옵니다.
Rawdata = pd.read_csv('dataset/mortgage.csv')
# Print Column names
print("'mortgage' data column name : ", list(Rawdata.columns.values))
# Allocate column index based on Input and Output varaibles

Input_Column_Index = range(0,15)
Target_Column_Index = np.array([15])

# 같은 데이터셋을 사용하기 위해서 난수를 고정합니다.
np.random.seed(100)
Train_Index = np.random.choice(np.shape(Rawdata)[0],int(np.shape(Rawdata)[0]*0.8),replace=False)

# Input variable과 Output variable을 Numpy array로 변환합니다.
Rawdata_Input = np.array(Rawdata)[:,Input_Column_Index]
Rawdata_Output = np.array(Rawdata)[:,Target_Column_Index]

# Training data와 Test data를 나누어 줍니다.
Train_Input = Rawdata_Input[Train_Index,:]
Train_Output = Rawdata_Output[Train_Index,:]
Test_Input = Rawdata_Input[np.delete(range(np.shape(Rawdata)[0]),Train_Index),:]
Test_Output = Rawdata_Output[np.delete(range(np.shape(Rawdata)[0]),Train_Index),:]
print('Data partition complete! \nTrain_Input shape :',np.shape(Train_Input),'\nTrain_Output shape :',np.shape(Train_Output))
print('Test_Input shape :',np.shape(Test_Input),'\nTest_Output shape :',np.shape(Test_Output))

def standardization(Data,Data2):
    return ((Data - np.mean(Data2, axis=0)) / np.std(Data2, axis=0))

Train_Input_Normalized = copy.deepcopy(standardization(Train_Input,Train_Input))
Test_Input_Normalized = copy.deepcopy(standardization(Test_Input,Train_Input))
print("standardization complete!")


# Best Hyperparameter를 찾기위하여 3-Fold Cross Validation을 한다
def k_Fold_Maker(InputData,OutputData,Partition_Number):
    Index = 0
    Input_List = list()
    Output_List = list()
    Length = int(np.floor(np.shape(InputData)[0]/Partition_Number))
    for i in range(Partition_Number):
        if(i == (Partition_Number-1)):
            Input_List.append(InputData[range(Index+(Length* i), np.shape(InputData)[0]), :])
            Output_List.append(OutputData[range(Index+(Length* i), np.shape(InputData)[0]), :])
        else:
            Input_List.append(InputData[range(Index + (Length * i), Index + (Length * (i + 1))), :])
            Output_List.append(OutputData[range(Index + (Length * i), Index + (Length * (i + 1))), :])
    return(Input_List,Output_List)


# Make 3-Fold dataset for Cross validation
Fold_Input, Fold_Output = k_Fold_Maker(Train_Input_Normalized, Train_Output, FOLD_VALUE)

# Cross validation을 위한 함수
def CV_Result_Each_Model(Hyper_Para,Model):
# 모든 결과를 담을 객체
    FULL_Results = list()
    for i in range(FOLD_VALUE):
        Tr_Index = np.delete(range(FOLD_VALUE),i) #Training 에 사용할 Fold Index
        Val_Index = i                    #Validation에 사용할 Fold Index

        TRAIN_INPUT  = list()
        TRAIN_OUTPUT = list()
        for j in Tr_Index:
            TRAIN_INPUT.append(Fold_Input[j])
            TRAIN_OUTPUT.append(Fold_Output[j])
        TRAIN_INPUT = np.concatenate(TRAIN_INPUT)
        TRAIN_OUTPUT = np.concatenate(TRAIN_OUTPUT)
        VALID_INPUT = Fold_Input[i]
        VALID_OUTPUT = Fold_Output[i]
        FULL_Results.append(Model(Hyper_Para,TRAIN_INPUT,TRAIN_OUTPUT,VALID_INPUT,VALID_OUTPUT))
    print("CV Complete!")
    FULL_Results=np.concatenate(FULL_Results,axis=0)
    return(FULL_Results)

'mortgage' data column name :  ['OneMonthCDRate', 'OneY.CMaturityRate', 'ThreeM.Rate.AuctionAverage', 'ThreeM.Rate.SecondaryMarket', 'ThreeY.CMaturityRate', 'FiveY.CMaturityRate', 'BankCredit', 'Currency', 'DemandDeposits', 'FederalFunds', 'MoneyStock', 'CheckableDeposits', 'LoansLeases', 'SavingsDeposits', 'TradeCurrencies', 'ThirtyY.CMortgageRate']
Data partition complete! 
Train_Input shape : (839, 15) 
Train_Output shape : (839, 1)
Test_Input shape : (210, 15) 
Test_Output shape : (210, 1)
standardization complete!


# Model1. Neural network
## Neuralnetwork 함수와 3Fold CV를 위해 함수를 생성 후 이행

In [3]:
def NeuralNetwork(HIDDEN_LAYER,Train_Input,TR_OUTPUT,Val_Input,Val_Output):
    MLP=MLPRegressor(hidden_layer_sizes=HIDDEN_LAYER,max_iter=2000 ,random_state =RANDOM_STATE).fit(Train_Input,TR_OUTPUT[:,0])
    Predict_Value = MLP.predict(Val_Input)
    return(np.concatenate((Predict_Value[:,np.newaxis],Val_Output),axis=1))


# 자 이제 NeuralNet을 기준으로 여러가지를 3-Fold CV를 해보도록 한다.
One_Layer_Node_10 = CV_Result_Each_Model([10],NeuralNetwork)
One_Layer_Node_20 = CV_Result_Each_Model([20],NeuralNetwork)
Two_Layer_Node_10 = CV_Result_Each_Model([10,10],NeuralNetwork)
Two_Layer_Node_20 = CV_Result_Each_Model([20,20],NeuralNetwork)
Three_Layer_Node_10 = CV_Result_Each_Model([10,10,10],NeuralNetwork)
Three_Layer_Node_20 = CV_Result_Each_Model([20,20,20],NeuralNetwork)

Results=pd.concat((Valid_Index(One_Layer_Node_10,'One_10'),
Valid_Index(One_Layer_Node_20,'One_20'),
Valid_Index(Two_Layer_Node_10,'Two_10'),
Valid_Index(Two_Layer_Node_20,'Two_20'),
Valid_Index(Three_Layer_Node_10,'Three_10'),
Valid_Index(Three_Layer_Node_20,'Three_20')))

CV Complete!
CV Complete!
CV Complete!
CV Complete!
CV Complete!
CV Complete!


In [4]:
ANN_Results = Results.sort_values(by=['MSE'],ascending=True)
ANN_Results

,MAE,MAPE,MSE,Model
0,0.156144,0.0234561,0.0490068,Three_20
0,0.156929,0.0215356,0.0493351,Two_20
0,0.204422,0.0306931,0.0741966,Three_10
0,0.219231,0.0306848,0.0822033,One_20
0,0.21358,0.0298132,0.0880638,Two_10
0,0.241195,0.0379738,0.0934222,One_10


# Model2. Bagging Neural network
## Neuralnetwork 함수와 3Fold CV를 위해 함수를 생성 후 이행

In [5]:
def B_NeuralNetwork(HIDDEN_LAYER,Train_Input,TR_OUTPUT,Val_Input,Val_Output):
    MLP=MLPRegressor(hidden_layer_sizes=HIDDEN_LAYER,max_iter=2000 ,random_state =RANDOM_STATE)
    BMLP = BaggingRegressor(n_estimators=30, base_estimator=MLP, 
                             random_state=RANDOM_STATE, n_jobs=-1).fit(Train_Input, TR_OUTPUT[:, 0])
    Predict_Value = BMLP.predict(Val_Input)
    return(np.concatenate((Predict_Value[:,np.newaxis],Val_Output),axis=1))

# 자 이제 NeuralNet을 기준으로 여러가지를 3-Fold CV를 해보도록 한다.
B_Three_Layer_Node_10 = CV_Result_Each_Model([10,10,10],B_NeuralNetwork)
B_Three_Layer_Node_20 = CV_Result_Each_Model([20,20,20],B_NeuralNetwork)

Results=pd.concat((Valid_Index(B_Three_Layer_Node_10,'B_Three_10'),
Valid_Index(B_Three_Layer_Node_20,'B_Three_20')))

CV Complete!
CV Complete!


In [6]:
B_ANN_Results = Results.sort_values(by=['MSE'],ascending=True)
B_ANN_Results

,MAE,MAPE,MSE,Model
0,0.138131,0.018773,0.0372675,B_Three_10
0,0.19032,0.0266004,0.0613137,B_Three_20


# Model3. RandomForest
## RandomForest 함수와 3Fold CV를 위해 함수를 생성 후 이행

In [7]:
def RandomForest(NUMBER,Train_Input,TR_OUTPUT,Val_Input,Val_Output):
    RF=RandomForestRegressor(n_estimators=NUMBER,
                           max_features="sqrt",
                           random_state=RANDOM_STATE).fit(Train_Input,TR_OUTPUT[:,0])
    Predict_Value = RF.predict(Val_Input)
    return(np.concatenate((Predict_Value[:,np.newaxis],Val_Output),axis=1))    

RF50_CV = CV_Result_Each_Model(50,RandomForest)
RF100_CV = CV_Result_Each_Model(100,RandomForest)
RF150_CV = CV_Result_Each_Model(150,RandomForest)
RF200_CV = CV_Result_Each_Model(200,RandomForest)

RF_3CV_Results=Valid_Index((RF50_CV),"RF50_CV").append([
Valid_Index((RF100_CV),"RF100_CV"),
Valid_Index((RF150_CV),"RF150_CV"),
Valid_Index((RF200_CV),"RF200_CV")])

RF_Results = RF_3CV_Results.sort_values(by=['MSE'],ascending=True)
RF_Results

CV Complete!
CV Complete!
CV Complete!
CV Complete!


,MAE,MAPE,MSE,Model
0,0.0961123,0.0118863,0.0278078,RF200_CV
0,0.0970845,0.0120079,0.0281231,RF150_CV
0,0.0976325,0.0121178,0.028453,RF100_CV
0,0.0992081,0.0122161,0.0295819,RF50_CV


# Model4. Bagging Tree
## Bagging Tree 함수와 3Fold CV를 위해 함수를 생성 후 이행

In [8]:
def B_Tree(NUMBER,Train_Input,TR_OUTPUT,Val_Input,Val_Output):
    Tree = DecisionTreeRegressor(max_depth=NUMBER, random_state=RANDOM_STATE).fit(Train_Input, TR_OUTPUT[:, 0])
    BMLP = BaggingRegressor(n_estimators=30, base_estimator=Tree, random_state=RANDOM_STATE, n_jobs=-1).fit(Train_Input, TR_OUTPUT[:, 0])
    Predict_Value = BMLP.predict(Val_Input)
    return(np.concatenate((Predict_Value[:,np.newaxis],Val_Output),axis=1))

# 자 이제 NeuralNet을 기준으로 여러가지를 3-Fold CV를 해보도록 한다.
B_Tree_D6 = CV_Result_Each_Model(6,B_Tree)
B_Tree_D5 = CV_Result_Each_Model(5,B_Tree)

Results=pd.concat((Valid_Index(B_Tree_D6,'B_Tree_D6'),
Valid_Index(B_Tree_D5,'B_Tree_D5')))

BT_3CV_Results = Results.sort_values(by=['MSE'],ascending=True)
BT_3CV_Results

CV Complete!
CV Complete!


,MAE,MAPE,MSE,Model
0,0.131925,0.017463,0.0405543,B_Tree_D6
0,0.150331,0.0203429,0.0486948,B_Tree_D5


# Model5. Gradient Boosting Machine
## Gradient Boosting Machine 함수와 3Fold CV를 위해 함수를 생성 후 이행

In [9]:
def GBM(NUMBER,Train_Input,TR_OUTPUT,Val_Input,Val_Output):
    gbm=GradientBoostingRegressor(n_estimators=NUMBER,random_state=RANDOM_STATE).fit(Train_Input,TR_OUTPUT[:,0])
    Predict_Value = gbm.predict(Val_Input)
    return(np.concatenate((Predict_Value[:,np.newaxis],Val_Output),axis=1))

GBM_100 = CV_Result_Each_Model(100,GBM)
GBM_150 = CV_Result_Each_Model(150,GBM)
GBM_200 = CV_Result_Each_Model(200,GBM)
GBM_250 = CV_Result_Each_Model(250,GBM)
GBM_300 = CV_Result_Each_Model(300,GBM)

GBM_3CV_Results=Valid_Index((GBM_100),"GBM_100").append([
Valid_Index((GBM_150),"GBM_150"),
Valid_Index((GBM_200),"GBM_200"),
Valid_Index((GBM_250),"GBM_250"),
Valid_Index((GBM_300),"GBM_300")])

GBM_3CV_Results = GBM_3CV_Results.sort_values(by=['MSE'],ascending=True)
GBM_3CV_Results

CV Complete!
CV Complete!
CV Complete!
CV Complete!
CV Complete!


,MAE,MAPE,MSE,Model
0,0.0988772,0.0124856,0.0265852,GBM_300
0,0.0999459,0.012639,0.0268782,GBM_250
0,0.101775,0.0128764,0.0273836,GBM_200
0,0.104558,0.0132602,0.0282485,GBM_150
0,0.108536,0.0138255,0.0295596,GBM_100
